In [1]:
import os

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

In [2]:
import pandas as pd
import glob

# Gabungkan semua file CSV menjadi satu DataFrame.
files = glob.glob(os.path.join(DATA_DIR, 'branch_*.csv'))
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs)

print(df)

   transaction_id branch        date  product_id  quantity  price customer_id
0               1      A  2023-01-01         101         2   50.0        C001
1               2      A  2023-01-02         102         1   20.0        C002
2               3      A  2023-01-03         103         3   15.0        C003
3               4      A  2023-01-04         101         2   50.0        C004
4               5      A         NaN         104         1   30.0        C005
5               6      A  2023-01-06         105         2   25.0         NaN
0               7      B  2023-01-01         101         1   50.0        C006
1               8      B  2023-01-02         106         2   40.0        C007
2               9      B  2023-01-03         107         3   25.0        C008
3              10      B  2023-01-04         108         1   30.0        C009
4              11      B  2023-01-05         109         2   35.0        C010
5              12      B  2023-01-06         107         3   25.

In [3]:
# 1. Hapus baris yang memiliki nilai NaN pada kolom transaction_id, date, dan customer_id.
df.dropna(subset=['transaction_id', 'date', 'customer_id'], inplace=True)

print(df)

   transaction_id branch        date  product_id  quantity  price customer_id
0               1      A  2023-01-01         101         2   50.0        C001
1               2      A  2023-01-02         102         1   20.0        C002
2               3      A  2023-01-03         103         3   15.0        C003
3               4      A  2023-01-04         101         2   50.0        C004
0               7      B  2023-01-01         101         1   50.0        C006
1               8      B  2023-01-02         106         2   40.0        C007
2               9      B  2023-01-03         107         3   25.0        C008
3              10      B  2023-01-04         108         1   30.0        C009
4              11      B  2023-01-05         109         2   35.0        C010
5              12      B  2023-01-06         107         3   25.0        C011
0              13      C  2023-01-01         110         1   60.0        C012
1              14      C  2023-01-02         111         2   20.

In [4]:
# 2. Ubah format kolom date menjadi tipe datetime.
df['date'] = pd.to_datetime(df['date'], errors='coerce')

print(df.dtypes)

transaction_id             int64
branch                    object
date              datetime64[ns]
product_id                 int64
quantity                   int64
price                    float64
customer_id               object
dtype: object


In [5]:
# 3. Hilangkan duplikat berdasarkan transaction_id, pilih data berdasarkan date terbaru.
df = df.sort_values('date').drop_duplicates('transaction_id', keep='last')

print(df)

   transaction_id branch       date  product_id  quantity  price customer_id
0               1      A 2023-01-01         101         2   50.0        C001
0               7      B 2023-01-01         101         1   50.0        C006
0              13      C 2023-01-01         110         1   60.0        C012
1               2      A 2023-01-02         102         1   20.0        C002
1               8      B 2023-01-02         106         2   40.0        C007
2               3      A 2023-01-03         103         3   15.0        C003
2               9      B 2023-01-03         107         3   25.0        C008
2              14      C 2023-01-03         112         1   70.0        C014
3               4      A 2023-01-04         101         2   50.0        C004
3              10      B 2023-01-04         108         1   30.0        C009
3              15      C 2023-01-04         113         2   50.0        C015
4              11      B 2023-01-05         109         2   35.0        C010

In [6]:
# 4. Hitung total penjualan per cabang
df['total'] = df['quantity'] * df['price']
total_sales_per_branch = df.groupby('branch')['total'].sum().reset_index()

print(total_sales_per_branch)

  branch  total
0      A  265.0
1      B  380.0
2      C  480.0


In [7]:
# Simpan hasilnya ke file baru total_sales_per_branch.csv dengan kolom branch dan total.
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_file = os.path.join(OUTPUT_DIR, 'total_sales_per_branch.csv')
total_sales_per_branch.to_csv(output_file, index=False)